In [1]:
import pandas as pd # Pour le dataframe
import numpy as np # Pour la normalisation et calculs de moyenne
import matplotlib.pyplot as plt # Pour la visualisation

from PIL import Image

import librosa # Pour l'extraction des features et la lecture des fichiers wav
import librosa.display # Pour récupérer les spectrogrammes des audio
import librosa.feature

import os # C'est ce qui va nous permettre d'itérer sur les fichiers de l'environnement de travail

from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import train_test_split, GridSearchCV, validation_curve, RandomizedSearchCV # Split de dataset et optimisation des hyperparamètres
from sklearn.ensemble import RandomForestClassifier # Random forest
from sklearn.ensemble import GradientBoostingClassifier # XGBoost
from sklearn.neighbors import KNeighborsClassifier # k-NN
from sklearn.svm import SVC # SVM
from sklearn.metrics import accuracy_score, confusion_matrix, recall_score, f1_score, zero_one_loss, classification_report # Métriques pour la mesure de performances
from sklearn.preprocessing import normalize, StandardScaler

import tensorflow as tf # Pour le reseau de neurones simple et pour le CNN

import seaborn as sns

from keras.models import Sequential
from keras.layers import Dense
from scikeras.wrappers import KerasClassifier
from keras.optimizers import Adam
from keras.utils import to_categorical

from xgboost import XGBClassifier
from pprint import pprint

In [37]:
# Définissons la liste avec les genres :

genres = ['blues', 'classical', 'country', 'disco', 'hiphop','jazz', 'metal', 'pop', 'reggae', 'rock']

# Création d'un dictionnaire avec les genres vide :

audio_files = {}


for g in genres:
  audio_files[g] = []

# Remplissage du dictionnaire en important les fichiers audio avec Librosa :

for g in genres:
  for audio in os.listdir(f'./Data/genres_test/{g}'):
    audio_files[g].append(librosa.load(f'./Data/genres_test/{g}/{audio}')[0])

In [38]:
def audio_pipeline(audio):


  features = []

  # Calcul du ZCR
  
  chroma_stft = librosa.feature.chroma_stft(y=audio)
  features.append(np.mean(chroma_stft))
  features.append(np.var(chroma_stft))
  
  rms = librosa.feature.rms(y=audio)
  features.append(np.mean(rms))
  features.append(np.var(rms))
  
  # Calcul de la moyenne du Spectral centroid
  
  # spectral_centroids = librosa.feature.spectral_centroid(y=audio)[0]
  spectral_centroids = librosa.feature.spectral_centroid(y=audio)
  features.append(np.mean(spectral_centroids))
  features.append(np.var(spectral_centroids))

  spectral_bandwidth = librosa.feature.spectral_bandwidth(y=audio)
  features.append(np.mean(spectral_bandwidth))
  features.append(np.var(spectral_bandwidth))

  rolloff = librosa.feature.spectral_rolloff(y=audio)
  features.append(np.mean(rolloff))
  features.append(np.var(rolloff))

  zcr = librosa.feature.zero_crossing_rate(y=audio)
  features.append(np.mean(zcr))
  features.append(np.var(zcr))
  
  harmony = librosa.effects.harmonic(y=audio)
  features.append(np.mean(harmony))
  features.append(np.var(harmony))
  
  tempo = librosa.feature.tempo(y=audio)
  features.append(tempo[0])

  # Calcul des moyennes des MFCC

  mfcc = librosa.feature.mfcc(y=audio)

  for x in mfcc:
    features.append(np.mean(x))
    features.append(np.var(x))


  return features

In [39]:
column_names = ['chroma_stft_mean','chroma_stft_var','rms_mean','rms_var', 
            'spectral_centroid_mean','spectral_centroid_var','spectral_bandwidth_mean',
            'spectral_bandwidth_var','rolloff_mean','rolloff_var','zero_crossing_rate_mean',
            'zero_crossing_rate_var','harmony_mean','harmony_var', 'tempo',
            'mfcc1_mean','mfcc1_var','mfcc2_mean','mfcc2_var','mfcc3_mean','mfcc3_var',
            'mfcc4_mean','mfcc4_var','mfcc5_mean','mfcc5_var','mfcc6_mean','mfcc6_var','mfcc7_mean',
            'mfcc7_var','mfcc8_mean','mfcc8_var','mfcc9_mean','mfcc9_var','mfcc10_mean','mfcc10_var',
            'mfcc11_mean','mfcc11_var','mfcc12_mean','mfcc12_var','mfcc13_mean','mfcc13_var',
            'mfcc14_mean','mfcc14_var','mfcc15_mean','mfcc15_var','mfcc16_mean','mfcc16_var',
            'mfcc17_mean','mfcc17_var','mfcc18_mean','mfcc18_var','mfcc19_mean','mfcc19_var',
            'mfcc20_mean','mfcc20_var', 'label']


# Création d'un dataframe vide

df = pd.DataFrame(columns = column_names)

# On itère sur les audios pour remplir le dataframe

i = 0
for g in genres:
  for music in audio_files[g]:
    # audio_pipeline(music)+[g]
    # break
    df.loc[i] = audio_pipeline(music)+[g]
    i+=1

In [40]:
df.to_csv('./content/music4.csv', index = False)